################################################################################
#Licensed Materials - Property of IBM
#(C) Copyright IBM Corp. 2020
#US Government Users Restricted Rights - Use, duplication disclosure restricted
#by GSA ADP Schedule Contract with IBM Corp.
################################################################################

The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below.
Specifically, the Source Components and Sample Materials clause included in the License Information document for
Watson Studio Auto-generated Notebook applies to the auto-generated notebooks. 
By downloading, copying, accessing, or otherwise using the materials, you agree to the License Terms.
http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BHU2B7&title=IBM%20Watson%20Studio%20Auto-generated%20Notebook%20V2.1

### IBM AutoAI Auto-Generated Notebook v1.13.1

**Note:** Notebook code generated using AutoAI will execute successfully. If code is modified or reordered,   
there is no guarantee it will successfully execute. This pipeline is optimized for the original dataset.  
The pipeline may fail or produce sub-optimium results if used with different data. For different data,  
please consider returning to AutoAI Experiments to generate a new pipeline. Please read our documentation   
for more information:   
(Cloud Platform) https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html .
(Cloud Pak For Data) https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/autoai-notebook.html .


Before modifying the pipeline or trying to re-fit the pipeline, consider:   
The notebook converts dataframes to numpy arrays before fitting the pipeline   
(a current restriction of the preprocessor pipeline). The known_values_list is passed by reference   
and populated with categorical values during fit of the preprocessing pipeline. Delete its members before re-fitting.

### Representing Pipeline_2 


### 1. Set Up
If lightgbm or xgboost installation fails, please follow:
 - [lightgbm docs](https://lightgbm.readthedocs.io/en/latest/Installation-Guide.html)
 - [xgboost docs](https://xgboost.readthedocs.io/en/latest/build.html)

In [1]:

try:
    import autoai_libs
except Exception as e:
    import subprocess
    out = subprocess.check_output('pip install autoai-libs'.split(' '))
    for line in out.splitlines():
        print(line)
    import autoai_libs
import sklearn
try:
    import xgboost
except:
    print('xgboost, if needed, will be installed and imported later')
try:
    import lightgbm
except:
    print('lightgbm, if needed, will be installed and imported later')
from sklearn.cluster import FeatureAgglomeration
import numpy
from numpy import inf, nan, dtype, mean
from autoai_libs.sklearn.custom_scorers import CustomScorers
import sklearn.ensemble
from autoai_libs.cognito.transforms.transform_utils import TExtras, FC
from autoai_libs.transformers.exportable import *
from autoai_libs.utils.exportable_utils import *
from sklearn.pipeline import Pipeline
known_values_list=[]

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200906121956-0003
KERNEL_ID = f3f23ea5-241d-4c83-b526-179a5086ffdd
lightgbm, if needed, will be installed and imported later


In [2]:
# compose a decorator to assist pipeline instantiation via import of modules and installation of packages
def decorator_retries(func):
    def install_import_retry(*args, **kwargs):
        retries = 0
        successful = False
        failed_retries = 0
        while retries < 100 and failed_retries < 10 and not successful:
            retries += 1
            failed_retries += 1
            try:
                result = func(*args, **kwargs)
                successful = True
            except Exception as e:
                estr = str(e)
                if estr.startswith('name ') and estr.endswith(' is not defined'):
                    try:
                        import importlib
                        module_name = estr.split("'")[1]
                        module = importlib.import_module(module_name)
                        globals().update({module_name: module})
                        print('import successful for ' + module_name)
                        failed_retries -= 1
                    except Exception as import_failure:
                        print('import of ' + module_name + ' failed with: ' + str(import_failure))
                        import subprocess
                        if module_name == 'lightgbm':
                            try:
                                print('attempting pip install of ' + module_name)
                                process = subprocess.Popen('pip install ' + module_name, shell=True)
                                process.wait()
                            except Exception as E:
                                print(E)
                                try:
                                    import sys
                                    print('attempting conda install of ' + module_name)
                                    process = subprocess.Popen('conda install --yes --prefix {sys.prefix} -c powerai ' + module_name, shell = True)
                                    process.wait()
                                except Exception as lightgbm_installation_error:
                                    print('lightgbm installation failed!' + lightgbm_installation_error)
                        else:
                            print('attempting pip install of ' + module_name)
                            process = subprocess.Popen('pip install ' + module_name, shell=True)
                            process.wait()
                        try:
                            print('re-attempting import of ' + module_name)
                            module = importlib.import_module(module_name)
                            globals().update({module_name: module})
                            print('import successful for ' + module_name)
                            failed_retries -= 1
                        except Exception as import_or_installation_failure:
                            print('failure installing and/or importing ' + module_name + ' error was: ' + str(
                                import_or_installation_failure))
                            raise (ModuleNotFoundError('Missing package in environment for ' + module_name +
                                                       '? Try import and/or pip install manually?'))
                elif type(e) is AttributeError:
                    if 'module ' in estr and ' has no attribute ' in estr:
                        pieces = estr.split("'")
                        if len(pieces) == 5:
                            try:
                                import importlib
                                print('re-attempting import of ' + pieces[3] + ' from ' + pieces[1])
                                module = importlib.import_module('.' + pieces[3], pieces[1])
                                failed_retries -= 1
                            except:
                                print('failed attempt to import ' + pieces[3])
                                raise (e)
                        else:
                            raise (e)
                else:
                    raise (e)
        if successful:
            print('Pipeline successfully instantiated')
        else:
            raise (ModuleNotFoundError(
                'Remaining missing imports/packages in environment? Retry cell and/or try pip install manually?'))
        return result
    return install_import_retry


### 2. Compose Pipeline

In [3]:
# metadata necessary to replicate AutoAI scores with the pipeline
_input_metadata = {'separator': ',', 'excel_sheet': 0, 'target_label_name': 'Final Hotspot', 'learning_type': 'classification', 'subsampling': None, 'pos_label': 1.0, 'pn': 'P2', 'cv_num_folds': 3, 'holdout_fraction': 0.1, 'optimization_metric': 'accuracy', 'random_state': 33, 'data_source': ''}

# define a function to compose the pipeline, and invoke it
@decorator_retries
def compose_pipeline():
    import numpy
    from numpy import nan, dtype, mean
    #
    # composing steps for toplevel Pipeline
    #
    _input_metadata = {'separator': ',', 'excel_sheet': 0, 'target_label_name': 'Final Hotspot', 'learning_type': 'classification', 'subsampling': None, 'pos_label': 1.0, 'pn': 'P2', 'cv_num_folds': 3, 'holdout_fraction': 0.1, 'optimization_metric': 'accuracy', 'random_state': 33, 'data_source': ''}
    steps = []
    steps.append(('column_selector', autoai_libs.transformers.exportable.ColumnSelector(activate_flag=True, columns_indices_list=[8, 9, 10, 11])))
    #
    # composing steps for preprocessor Pipeline
    #
    preprocessor__input_metadata = None
    preprocessor_steps = []
    #
    # composing steps for preprocessor_features FeatureUnion
    #
    preprocessor_features_transformer_list = []
    #
    # composing steps for preprocessor_features_categorical Pipeline
    #
    preprocessor_features_categorical__input_metadata = None
    preprocessor_features_categorical_steps = []
    preprocessor_features_categorical_steps.append(('cat_column_selector', autoai_libs.transformers.exportable.NumpyColumnSelector(columns=[2, 3])))
    preprocessor_features_categorical_steps.append(('cat_compress_strings', autoai_libs.transformers.exportable.CompressStrings(activate_flag=True, compress_type='hash', dtypes_list=['int_num', 'int_num'], missing_values_reference_list=['', '-', '?', nan], misslist_list=[[], []])))
    preprocessor_features_categorical_steps.append(('cat_missing_replacer', autoai_libs.transformers.exportable.NumpyReplaceMissingValues(filling_values=nan, missing_values=[])))
    preprocessor_features_categorical_steps.append(('cat_unknown_replacer', autoai_libs.transformers.exportable.NumpyReplaceUnknownValues(filling_values=nan, filling_values_list=[nan, nan], known_values_list=[[0, 1, 2, 3], [0, 1, 2, 3]], missing_values_reference_list=['', '-', '?', nan])))
    preprocessor_features_categorical_steps.append(('boolean2float_transformer', autoai_libs.transformers.exportable.boolean2float(activate_flag=True)))
    preprocessor_features_categorical_steps.append(('cat_imputer', autoai_libs.transformers.exportable.CatImputer(activate_flag=True, missing_values=nan, sklearn_version_family='20', strategy='most_frequent')))
    preprocessor_features_categorical_steps.append(('cat_encoder', autoai_libs.transformers.exportable.CatEncoder(activate_flag=True, categories='auto', dtype=numpy.float64, encoding='ordinal', handle_unknown='error', sklearn_version_family='20')))
    preprocessor_features_categorical_steps.append(('float32_transformer', autoai_libs.transformers.exportable.float32_transform(activate_flag=True)))
    # assembling preprocessor_features_categorical_ Pipeline
    preprocessor_features_categorical_pipeline = sklearn.pipeline.Pipeline(steps=preprocessor_features_categorical_steps)
    preprocessor_features_transformer_list.append(('categorical', preprocessor_features_categorical_pipeline))
    #
    # composing steps for preprocessor_features_numeric Pipeline
    #
    preprocessor_features_numeric__input_metadata = None
    preprocessor_features_numeric_steps = []
    preprocessor_features_numeric_steps.append(('num_column_selector', autoai_libs.transformers.exportable.NumpyColumnSelector(columns=[0, 1])))
    preprocessor_features_numeric_steps.append(('num_floatstr2float_transformer', autoai_libs.transformers.exportable.FloatStr2Float(activate_flag=True, dtypes_list=['float_num', 'float_num'], missing_values_reference_list=[])))
    preprocessor_features_numeric_steps.append(('num_missing_replacer', autoai_libs.transformers.exportable.NumpyReplaceMissingValues(filling_values=nan, missing_values=[])))
    preprocessor_features_numeric_steps.append(('num_imputer', autoai_libs.transformers.exportable.NumImputer(activate_flag=True, missing_values=nan, strategy='median')))
    preprocessor_features_numeric_steps.append(('num_scaler', autoai_libs.transformers.exportable.OptStandardScaler(num_scaler_copy=None, num_scaler_with_mean=None, num_scaler_with_std=None, use_scaler_flag=False)))
    preprocessor_features_numeric_steps.append(('float32_transformer', autoai_libs.transformers.exportable.float32_transform(activate_flag=True)))
    # assembling preprocessor_features_numeric_ Pipeline
    preprocessor_features_numeric_pipeline = sklearn.pipeline.Pipeline(steps=preprocessor_features_numeric_steps)
    preprocessor_features_transformer_list.append(('numeric', preprocessor_features_numeric_pipeline))
    # assembling preprocessor_features_ FeatureUnion
    preprocessor_features_pipeline = sklearn.pipeline.FeatureUnion(transformer_list=preprocessor_features_transformer_list)
    preprocessor_steps.append(('features', preprocessor_features_pipeline))
    preprocessor_steps.append(('permuter', autoai_libs.transformers.exportable.NumpyPermuteArray(axis=0, permutation_indices=[2, 3, 0, 1])))
    # assembling preprocessor_ Pipeline
    preprocessor_pipeline = sklearn.pipeline.Pipeline(steps=preprocessor_steps)
    steps.append(('preprocessor', preprocessor_pipeline))
    steps.append(('estimator', xgboost.sklearn.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1, gamma=0.9896322970127698, learning_rate=0.02011107259785676, max_delta_step=0, max_depth=2, min_child_weight=2, missing=None, n_estimators=703, n_jobs=1, nthread=None, objective='binary:logistic', random_state=33, reg_alpha=0.9663465441967188, reg_lambda=0.9974672124264845, scale_pos_weight=0.6414501799308605, seed=None, silent=True, subsample=0.9784057235273806, verbosity=0, tree_method='hist')))
    # assembling  Pipeline
    pipeline = sklearn.pipeline.Pipeline(steps=steps)
    return pipeline
pipeline = compose_pipeline()


Pipeline successfully instantiated


### 3. Extract needed parameter values from AutoAI run metadata

In [4]:

# Metadata used in retrieving data and computing metrics.  Customize as necessary for your environment.
#data_source='replace_with_path_and_csv_filename'
target_label_name = _input_metadata['target_label_name']
learning_type = _input_metadata['learning_type']
optimization_metric = _input_metadata['optimization_metric']
random_state = _input_metadata['random_state']
cv_num_folds = _input_metadata['cv_num_folds']
holdout_fraction = _input_metadata['holdout_fraction']
if 'data_provenance' in _input_metadata:
    data_provenance = _input_metadata['data_provenance']
else:
    data_provenance = None
if 'pos_label' in _input_metadata and learning_type == 'classification':
    pos_label = _input_metadata['pos_label']
else:
    pos_label = None


### 4. Create dataframe from dataset in Cloud Object Storage

In [5]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_0 = {
    'ENDPOINT': 'https://s3.eu-geo.objectstorage.softlayer.net',
    'IBM_AUTH_ENDPOINT': 'https://iam.bluemix.net/oidc/token/',
    'APIKEY': '9QmAEA4x4E1-fS1dLkw8GUnWrkhxBPvrBUvlhd2iurAX',
    'BUCKET': 'hackathon1-donotdelete-pr-ro2ad1pzg2uhme',
    'FILE': 'Hack_Preprocessed_DataSet_final123.csv',
    'SERVICE_NAME': 's3',
    'ASSET_ID': '1',
    }


In [6]:
#  Read the data as a dataframe
import pandas as pd

csv_encodings=['UTF-8','Latin-1'] # supplement list of encodings as necessary for your data
df = None
readable = None  # if automatic detection fails, you can supply a filename here

# First, obtain a readable object
# Cloud Object Storage data access
# Assumes COS credentials are in a dictionary named 'credentials_0'
   
credentials = df = globals().get('credentials_0')       
if readable is None and credentials is not None :
    try:
        import types
        import pandas as pd
        import io
    except Exception as import_exception:
        print('Error with importing packages - check if you installed them on your environment')
    try:
        if credentials['SERVICE_NAME'] == 's3':
            try:
                from botocore.client import Config
                import ibm_boto3
            except Exception as import_exception:
                print('Installing required packages!')
                !pip install ibm-cos-sdk
                print('accessing data via Cloud Object Storage')
            try:
                client = ibm_boto3.client(service_name=credentials['SERVICE_NAME'],
                                    ibm_api_key_id=credentials['APIKEY'],
                                    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
                                    config=Config(signature_version='oauth'),
                                    endpoint_url=credentials['ENDPOINT'])
            except Exception as cos_exception:
                print('unable to create client for cloud object storage')
            try:
                readable = client.get_object(Bucket=credentials['BUCKET'],Key=credentials['FILE'])['Body']
                # add missing __iter__ method, so pandas accepts readable as file-like object
                if not hasattr(readable, "__iter__"): readable.__iter__ = types.MethodType( __iter__, readable )
            except Exception as cos_access_exception:
                print('unable to access data object in cloud object storage with credentials supplied')
        elif credentials['SERVICE_NAME'] == 'fs':
            print('accessing data via File System')
            try:
                if credentials['FILE'].endswith('xlsx') or credentials['FILE'].endswith('xls'):
                    df = pd.read_excel(credentials['FILE'])
                else:
                    df = pd.read_csv(credentials['FILE'], sep = _input_metadata['separator'])
            except Exception as FS_access_exception:
                print('unable to access data object in File System with path supplied') 
    except Exception as data_access_exception:
        print('unable to access data object with credentials supplied') 

# IBM Cloud Pak for Data data access
project_filename = globals().get('project_filename')       
if readable is None and 'credentials_0' in globals() and 'ASSET_ID' in credentials_0:
    project_filename = credentials_0['ASSET_ID']
if project_filename != None and project_filename != '1':
    print('attempting project_lib access to ' + str(project_filename))
    try:
        from project_lib import Project
        project = Project.access()
        storage_credentials = project.get_storage_metadata()
        readable = project.get_file(project_filename)
    except Exception as project_exception:
        print('unable to access data using the project_lib interface and filename supplied')

# Use data_provenance as filename if other access mechanisms are unsuccessful
if readable is None and type(data_provenance) is str:
    print('attempting to access local file using path and name ' + data_provenance)
    readable = data_provenance

# Second, use pd.read_csv to read object, iterating over list of csv_encodings until successful
if readable is not None:
    for encoding in csv_encodings:
        try:
            if credentials['FILE'].endswith('xlsx') or credentials['FILE'].endswith('xls'):
                buffer = io.BytesIO(readable.read())
                buffer.seek(0)
                df = pd.read_excel(buffer, encoding=encoding,sheet_name=_input_metadata['excel_sheet'])
            else:
                df = pd.read_csv(readable, encoding = encoding, sep = _input_metadata['separator'])
            print('successfully loaded dataframe using encoding = ' + str(encoding))
            break
        except Exception as exception_dataread:
            print('unable to read csv using encoding ' + str(encoding))
            print('handled error was ' + str(exception_dataread))
    if df is None:
        print('unable to read file/object as a dataframe using supplied csv_encodings ' + str(csv_encodings))
        print(f'Please use \'insert to code\' on data panel to load dataframe.')
        raise(ValueError('unable to read file/object as a dataframe using supplied csv_encodings ' + str(csv_encodings)))

if isinstance(df,pd.DataFrame):
    print('Data loaded succesfully')
    if _input_metadata.get('subsampling') is not None:
        df = df.sample(frac=_input_metadata['subsampling'], random_state=_input_metadata['random_state']) if _input_metadata['subsampling'] <= 1.0 else df.sample(n=_input_metadata['subsampling'], random_state=_input_metadata['random_state'])
else:
    print('Data cannot be loaded with credentials supplied, please provide DataFrame with training data.')

successfully loaded dataframe using encoding = UTF-8
Data loaded succesfully


### 5. Preprocess Data

In [7]:
# Drop rows whose target is not defined
target = target_label_name # your target name here
if learning_type == 'regression':
    df[target] = pd.to_numeric(df[target], errors='coerce')
df.dropna('rows', how='any', subset=[target], inplace=True)


In [8]:
# extract X and y
df_X = df.drop(columns=[target])
df_y = df[target]


In [9]:
# Detach preprocessing pipeline (which needs to see all training data)
preprocessor_index = -1
preprocessing_steps = [] 
for i, step in enumerate(pipeline.steps):
    preprocessing_steps.append(step)
    if step[0]=='preprocessor':
        preprocessor_index = i
        break
#if len(pipeline.steps) > preprocessor_index+1 and pipeline.steps[preprocessor_index + 1][0] == 'cognito':
    #preprocessor_index += 1
    #preprocessing_steps.append(pipeline.steps[preprocessor_index])
if preprocessor_index >= 0:
    preprocessing_pipeline = Pipeline(memory=pipeline.memory, steps=preprocessing_steps)
    pipeline = Pipeline(steps=pipeline.steps[preprocessor_index+1:])

In [10]:
# Preprocess X
# preprocessor should see all data for cross_validate on the remaining steps to match autoai scores
known_values_list.clear()  #  known_values_list is filled in by the preprocessing_pipeline if needed
preprocessing_pipeline.fit(df_X.values, df_y.values)
X_prep = preprocessing_pipeline.transform(df_X.values)

### 6. Split data into Training and Holdout sets

In [11]:
# determine learning_type and perform holdout split (stratify conditionally)
if learning_type is None:
    # When the problem type is not available in the metadata, use the sklearn type_of_target to determine whether to stratify the holdout split
    # Caution:  This can mis-classify regression targets that can be expressed as integers as multiclass, in which case manually override the learning_type
    from sklearn.utils.multiclass import type_of_target
    if type_of_target(df_y.values) in ['multiclass', 'binary']:
        learning_type = 'classification'
    else:
        learning_type = 'regression'
    print('learning_type determined by type_of_target as:',learning_type)
else:
    print('learning_type specified as:',learning_type)
    
from sklearn.model_selection import train_test_split
if learning_type == 'classification':
    X, X_holdout, y, y_holdout = train_test_split(X_prep, df_y.values, test_size=holdout_fraction, random_state=random_state, stratify=df_y.values)
else:
    X, X_holdout, y, y_holdout = train_test_split(X_prep, df_y.values, test_size=holdout_fraction, random_state=random_state)


learning_type specified as: classification


#### 7. Generate features via Feature Engineering pipeline

In [12]:
#Detach Feature Engineering pipeline if next, fit it, and transform the training data
fe_pipeline = None
if pipeline.steps[0][0] == 'cognito':
    try:
        fe_pipeline = Pipeline(steps=[pipeline.steps[0]])
        X = fe_pipeline.fit_transform(X, y)
        X_holdout = fe_pipeline.transform(X_holdout)
        pipeline.steps = pipeline.steps[1:]
    except IndexError:
        try:
            print('Trying to compose pipeline with some of cognito steps')
            fe_pipeline = Pipeline(steps = list([pipeline.steps[0][1].steps[0],pipeline.steps[0][1].steps[1]]))
            X = fe_pipeline.fit_transform(X, y)
            X_holdout = fe_pipeline.transform(X_holdout)
            pipeline.steps = pipeline.steps[1:]
        except IndexError:
            print('Composing pipeline without cognito steps!')
            pipeline.steps = pipeline.steps[1:]


 ### 8. Additional setup: Define a function that returns a scorer for the target's positive label

In [13]:
# create a function to produce a scorer for a given positive label
def make_pos_label_scorer(scorer, pos_label):
    kwargs = {'pos_label':pos_label}
    for prop in ['needs_proba', 'needs_threshold']:
        if prop+'=True' in scorer._factory_args():
            kwargs[prop] = True
    if scorer._sign == -1:
        kwargs['greater_is_better'] = False
    from sklearn.metrics import make_scorer
    scorer=make_scorer(scorer._score_func, **kwargs)
    return scorer

### 9. Fit pipeline, predict on Holdout set, calculate score, perform cross-validation

In [14]:
# fit the remainder of the pipeline on the training data
pipeline.fit(X,y)

Pipeline(steps=[('estimator',
                 XGBClassifier(gamma=0.9896322970127698,
                               learning_rate=0.02011107259785676, max_depth=2,
                               min_child_weight=2, n_estimators=703,
                               random_state=33, reg_alpha=0.9663465441967188,
                               reg_lambda=0.9974672124264845,
                               scale_pos_weight=0.6414501799308605, silent=True,
                               subsample=0.9784057235273806, tree_method='hist',
                               verbosity=0))])

In [15]:
# predict on the holdout data
y_pred = pipeline.predict(X_holdout)

print("X=%s, Predicted=%s" % (X_holdout[0], y_pred[0]))
print("X=%s, Predicted=%s" % (X_holdout[1], y_pred[1]))
print("X=%s, Predicted=%s" % (X_holdout[2], y_pred[2]))
print("X=%s, Predicted=%s" % (X_holdout[3], y_pred[3]))
print("X=%s, Predicted=%s" % (X_holdout[4], y_pred[4]))
print("X=%s, Predicted=%s" % (X_holdout[5], y_pred[5]))
print("X=%s, Predicted=%s" % (X_holdout[6], y_pred[6]))
print("X=%s, Predicted=%s" % (X_holdout[7], y_pred[7]))
print("X=%s, Predicted=%s" % (X_holdout[8], y_pred[8]))
print("X=%s, Predicted=%s" % (X_holdout[9], y_pred[9]))
print("X=%s, Predicted=%s" % (X_holdout[10], y_pred[10]))
print("X=%s, Predicted=%s" % (X_holdout[11], y_pred[11]))
print("X=%s, Predicted=%s" % (X_holdout[12], y_pred[12]))
print("X=%s, Predicted=%s" % (X_holdout[13], y_pred[13]))
print("X=%s, Predicted=%s" % (X_holdout[14], y_pred[14]))
print("X=%s, Predicted=%s" % (X_holdout[15], y_pred[15]))
print("X=%s, Predicted=%s" % (X_holdout[16], y_pred[16]))
print("X=%s, Predicted=%s" % (X_holdout[17], y_pred[17]))

X=[4.1        0.05290909 3.         1.        ], Predicted=1
X=[4.3        0.14088051 2.         2.        ], Predicted=1
X=[0.87       0.05112479 2.         2.        ], Predicted=1
X=[4.1        0.14379747 3.         0.        ], Predicted=1
X=[3.67      0.0363447 2.        1.       ], Predicted=0
X=[4.1        0.04420346 2.         3.        ], Predicted=1
X=[1.56      0.0736745 2.        3.       ], Predicted=1
X=[3.6        0.13157895 1.         2.        ], Predicted=1
X=[3.2        0.04797185 2.         0.        ], Predicted=0
X=[3.2        0.05521473 1.         2.        ], Predicted=1
X=[1.56       0.09540441 1.         2.        ], Predicted=1
X=[4.2        0.04957908 1.         2.        ], Predicted=1
X=[2.98       0.16666667 3.         2.        ], Predicted=1
X=[2.98       0.02216548 3.         3.        ], Predicted=1
X=[4.6        0.08453441 3.         3.        ], Predicted=1
X=[2.75       0.02992838 2.         0.        ], Predicted=0
X=[3.67       0.07548077 2.     

In [16]:
# compute score for the optimization metric
# scorer may need pos_label, but not all scorers take pos_label parameter
from sklearn.metrics import get_scorer
scorer = get_scorer(optimization_metric)
score = None
#score = scorer(pipeline, X_holdout, y_holdout)  # this would suffice for simple cases
pos_label = None  # if you want to supply the pos_label, specify it here
if pos_label is None and 'pos_label' in _input_metadata:
    pos_label=_input_metadata['pos_label']
try:
    score = scorer(pipeline, X_holdout, y_holdout)
except Exception as e1:
    if learning_type is "classification" and (pos_label is None or str(pos_label)==''):
        print('You may have to provide a value for pos_label in order for a score to be calculated.')
        raise(e1)
    else:
        exception_string=str(e1)
        if 'pos_label' in exception_string:
            try:
                scorer = make_pos_label_scorer(scorer, pos_label=pos_label)
                score = scorer(pipeline, X_holdout, y_holdout)
                print('Retry was successful with pos_label supplied to scorer')
            except Exception as e2:
                print('Initial attempt to use scorer failed.  Exception was:')
                print(e1)
                print('')
                print('Retry with pos_label failed.  Exception was:')
                print(e2)
        else:
            raise(e1)

if score is not None:
    print(score)

0.8727272727272727


In [17]:
# cross_validate pipeline using training data
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold, KFold
if learning_type == 'classification':
    fold_generator = StratifiedKFold(n_splits=cv_num_folds, random_state=random_state)
else:
    fold_generator = KFold(n_splits=cv_num_folds, random_state=random_state)
cv_results = cross_validate(pipeline, X, y, cv=fold_generator, scoring={optimization_metric:scorer}, return_train_score=True)
import numpy as np
np.mean(cv_results['test_' + optimization_metric])

/home/spark/shared/user-libs/python3.6/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.8747759347623015

In [18]:
cv_results

{'fit_time': array([0.09579229, 0.09701514, 0.09791374]),
 'score_time': array([0.00159478, 0.00186801, 0.0015645 ]),
 'test_accuracy': array([0.85889571, 0.87654321, 0.88888889]),
 'train_accuracy': array([0.88271605, 0.88615385, 0.87384615])}

In [19]:
import ibm_db

#Enter the values for you database connection
dsn_driver = "DASHDB"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_hostname = "dashdb-txn-sbox-yp-lon02-15.services.eu-gb.bluemix.net" # e.g.: "awh-yp-small03.services.dal.bluemix.net"
dsn_port = "50000"                # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_uid = "wdp60370"        # e.g. "dash104434"
dsn_pwd = "j1^496bvwbvnmq18"       # e.g. "7dBZ3jWt9xN6$o0JiX!m"

In [20]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_36060ea8de5843bf8f2776d17d73596f = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='zgb8im85qjmJFKKSbMrDFjgn3wpQprGwZare4h8M_W6B',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_36060ea8de5843bf8f2776d17d73596f.get_object(Bucket='hackathon1-donotdelete-pr-ro2ad1pzg2uhme',Key='test_data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

,Release,Functionality Name,Functionality Num,CP,Efforts (Days),Complexity Factor,# of Defects,Module complexity,Weighted DD,Average Severity,Total Business Impact,Final Hotspot
0,31,Breach Details Send Email,162,39,43.68,0.600,5,0.66,0.125641,1,1,NaN
1,31,RCM Management,217,10,150.08,1.200,5,1.50,0.057237,2,2,NaN
2,31,Geography Popup - Mark up,154,11,101.76,0.600,3,3.67,0.031250,1,1,NaN
3,31,Geography Popup - Zoom in,261,22,29.70,1.500,2,0.87,0.090909,3,3,NaN
4,31,Check Breach Issues,215,31,39.68,2.375,3,1.98,0.096774,3,3,NaN


In [21]:
dsn = (
    "DRIVER=DASHDB;"
    "DATABASE=BLUDB;"
    "HOSTNAME=dashdb-txn-sbox-yp-lon02-15.services.eu-gb.bluemix.net;"
    "PORT=50000;"
    "PROTOCOL=TCPIP;"
    "UID=wdp60370;"
    "PWD={4};").format(dsn_database, dsn_hostname, dsn_port, dsn_uid, dsn_pwd)

conn = ibm_db.connect(dsn, "", "")

In [22]:
My_data = df_data_1.iloc[:, [7,8,9,10]].values
print(My_data)


# predict on the holdout data
y_pred = pipeline.predict(My_data)

print("X=%s, Predicted=%s" % (My_data[0], y_pred[0]))
print("X=%s, Predicted=%s" % (My_data[1], y_pred[1]))

[[0.66       0.12564103 1.         1.        ]
 [1.5        0.05723684 2.         2.        ]
 [3.67       0.03125    1.         1.        ]
 [0.87       0.09090909 3.         3.        ]
 [1.98       0.09677419 3.         3.        ]
 [0.87       0.01       1.         0.        ]]
X=[0.66       0.12564103 1.         1.        ], Predicted=1
X=[1.5        0.05723684 2.         2.        ], Predicted=1


In [23]:
print(y_pred)

[1 1 0 1 1 0]


In [24]:
for i in df_data_1.index:
    df_data_1.at[i, "Final Hotspot"] = y_pred[i]
        #df_data_1['Final Hotspot'] = y_pred[0]

In [25]:
from project_lib import Project
    
project = Project(sc,"5899a147-737e-48e1-86e6-7437f1f1ce71","p-14d6418677458d65b0995d7f0b3bc81cb3ad3163")
    
project.save_data(file_name = "Result"".csv",data = df_data_1.to_csv(index=False),overwrite=True)

{'file_name': 'Result.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'hackathon1-donotdelete-pr-ro2ad1pzg2uhme',
 'asset_id': '91eb67c8-a807-4c35-8198-0558a2d75ac5'}